### Pull Weather Data from API 
#### Raleigh, Charlotte, DC 1946 - 2015

In [1]:
import numpy as np
import json
import requests
import pandas as pd

#Sites we will loop through
site_name=['RDU','Reagan_Intl', 'CLT']
site_id=['GHCND:USW00013722','GHCND:USW00013743', 'GHCND:USW00013881']

### Pull data for a specific location for 10 year interval (max allowed by API in one call)

In [2]:
def return_results(url):
    
    #Create dictionaries for variable
    columns=['Year','SNOW','TAVG','CLDD','DX32','TMIN','TMAX','HTDD','DX90','DP01', 'DP05','DT32','FZF0','PRCP']
    all_col={}
    for column in columns:
        all_col[column] = []
   
    #Call API
    #headers = {'token': 'dKuJxEKlhIJuoZSjvKULivIPXWsRqspt' } #Two api keys, 1000 request limit/day
    headers = {'token': 'HGQwYMutaGbKyJUqOZchjtieVMKpMMTE'}
    response = requests.get(url, headers=headers)
    parsed=json.loads(response.text)   
       
    #list_vars=[SNOW,TAVG,CLDD,DX32,TMIN,TMAX,HTDD,DX90,DP01, DP05,DT32,FZF0,PRCP]
    for record in parsed['results']:
        for i in range(1,len(columns)):
            if record['datatype'] == columns[i]: 
                if i==1:
                    all_col['Year'].append(record['date'])
                all_col[columns[i]].append(record['value'])
               
    return all_col             

### Pull data in 10 year intervals for a specific site and merge results

In [3]:
def year_loop(start,end, by, code):
    
    columns=['Year','SNOW','TAVG','CLDD','DX32','TMIN','TMAX',
             'HTDD','DX90','DP01', 'DP05','DT32','FZF0','PRCP']
    all_col2={}
    for column in columns:
        all_col2[column] = []
        
    url='http://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GSOY&stationid=id_code&startdate=1946-01-01&enddate=1955-01-01&limit=1000'
    url=url.replace('id_code', code)

    ranges=np.arange(start, end+1, by)
    
    for i in range(len(ranges)-1):
        
        all_cols = return_results(url)

        for key, value in all_cols.items():
            all_col2[key].extend(value)
            #print(key, value)
        
        if i==6:
            pass

        else:
            url=url.replace(str(ranges[i+1]-1),str(ranges[i+2]-1))
            url=url.replace(str(ranges[i]),str(ranges[i+1]))
        
    all_col2=pd.DataFrame.from_dict(all_col2)
    all_col2['Year']=all_col2['Year'].map(lambda x: x[:4])

    return all_col2

### Pull data for Raleigh, Charlotte and DC from 1946 - 2010 - Load to CAS 

In [4]:
loop_df_rdu = year_loop(1946,2016,10,site_id[0])
loop_df_rdu['Location']=site_name[0]

loop_df_rea=year_loop(1946,2016,10,site_id[1])
loop_df_rea['Location']=site_name[1]

loop_df_clt=year_loop(1946,2016,10,site_id[2])
loop_df_clt['Location']=site_name[2]

API_Data=pd.concat([loop_df_rdu, loop_df_rea, loop_df_clt], axis=0)

In [5]:
API_Data.head()

,CLDD,DP01,DP05,DT32,DX32,DX90,FZF0,HTDD,PRCP,SNOW,TAVG,TMAX,TMIN,Year,Location
0,721.9,131,35,50,0,28,0.0,1834.8,1280.4,138,15.82,21.95,9.68,1946,RDU
1,855.3,133,28,92,3,44,0.0,1905.6,1168.2,99,14.83,20.89,8.77,1947,RDU
2,856.4,149,39,61,8,49,0.0,1978.7,1327.1,622,15.48,21.72,9.23,1948,RDU
3,903.8,126,27,58,0,44,-1.1,1611.2,1106.1,0,16.33,22.36,10.29,1949,RDU
4,840.5,125,22,77,1,44,0.0,1709.7,962.2,8,15.47,21.78,9.16,1950,RDU
